## MEET MANISH VORA
## NIRMA UNIVERSITY
## meetvora090201@gmail.com

## ELI LILLY DATA SCIENCE COMPETITON

In [30]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [31]:
path = r"C:\Users\MEET VORA\exam\dataset\train.csv"
path1 = r"C:\Users\MEET VORA\exam\dataset\test.csv"
path2 = r"C:\Users\MEET VORA\exam\dataset\sample_submission.csv"

In [32]:
data_frame = pd.read_csv(path)
data_frame.head()

,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence,music_genre
0,MSC_83537.0,Estrellitas y Duendes,49.178,0.970522,0.580508,214625.776,0.192107,0.000000,Scale E,0.147134,-14.140,NaN,Major,0.046404,143.78799999999998,Band,0.598965,Jazz
1,MSC_22044.0,Al Norte,59.827,1.009380,0.687542,216232.195,0.265942,0.000031,Scale A,0.174655,-13.716,Male,?,0.039949,?,Band,0.357194,Jazz
2,MSC_62017.0,Yeah! (feat. Lil Jon & Ludacris),89.023,0.020041,0.967948,273314.723,0.857411,0.000000,Scale D,0.042030,-4.995,Female,Major,0.119917,105.01799999999999,Band,0.635525,Rap
3,MSC_76365.0,Can’t You See,55.762,0.010284,0.616287,189189.605,0.970513,0.000220,Scale D,0.124637,-4.262,Both,Major,0.167493,?,Duet,0.983750,Rock
4,MSC_71493.0,"Sonata III (G Moll), BWV 1029: Adagio",45.095,0.858769,0.280645,410136.987,0.114732,0.002753,Scale F Sharp,0.134782,-26.922,Female,Minor,0.039139,112.18299999999999,Band,0.074412,Classical


In [33]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15681 entries, 0 to 15680
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instance_id        15681 non-null  object 
 1   track_name         15681 non-null  object 
 2   popularity         15039 non-null  float64
 3   acousticness       15681 non-null  float64
 4   danceability       15125 non-null  float64
 5   duration_ms        15587 non-null  float64
 6   energy             15587 non-null  float64
 7   instrumentalness   15586 non-null  float64
 8   key                15681 non-null  object 
 9   liveness           15681 non-null  float64
 10  loudness           15645 non-null  float64
 11  voice_gender       14916 non-null  object 
 12  mode               15646 non-null  object 
 13  speechiness        15655 non-null  float64
 14  tempo              15681 non-null  object 
 15  musician_category  14321 non-null  object 
 16  valence            150

In [34]:
print("shape of dataframe : ",data_frame.shape)

shape of dataframe :  (15681, 18)


In [35]:
data_frame = data_frame.drop(columns=['instance_id','track_name','voice_gender','musician_category'])

In [36]:
#data_frame = data_frame.dropna()
print("shape of dataframe : ",data_frame.shape)

shape of dataframe :  (15681, 14)


In [37]:
data_frame = data_frame.drop_duplicates()
print("shape of dataframe : ",data_frame.shape)

shape of dataframe :  (15681, 14)


In [38]:
X = data_frame.drop(columns=['music_genre'])
y = data_frame.drop(data_frame.iloc[:, :13],axis = 1)
print("X shape : ",X.shape)
print("y shape : ",y.shape)

X shape :  (15681, 13)
y shape :  (15681, 1)


In [39]:
category = np.array(y.music_genre.unique())
print(category)

['Jazz' 'Rap' 'Rock' 'Classical' 'Electronic' 'Country' 'Hip-Hop']


In [40]:
d = defaultdict(LabelEncoder)

def encode(col):
    if col.dtype == 'object':
        return d[str(col)].fit_transform(col)
    else:
        return col 
    
X = X.apply(encode)
y = y.apply(encode)

In [41]:
#X['loudness'] = X['loudness']*-1
X

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,49.178,0.970522,0.580508,214625.776,0.192107,0.000000,7,0.147134,-14.140,1,0.046404,5291,0.598965
1,59.827,1.009380,0.687542,216232.195,0.265942,0.000031,0,0.174655,-13.716,0,0.039949,12331,0.357194
2,89.023,0.020041,0.967948,273314.723,0.857411,0.000000,5,0.042030,-4.995,1,0.119917,615,0.635525
3,55.762,0.010284,0.616287,189189.605,0.970513,0.000220,5,0.124637,-4.262,1,0.167493,12331,0.983750
4,45.095,0.858769,0.280645,410136.987,0.114732,0.002753,9,0.134782,-26.922,2,0.039139,1466,0.074412
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15676,0.000,0.866988,0.387476,-1.000,0.196799,0.028725,2,0.248399,-16.331,1,0.045251,2822,0.624518
15677,40.514,0.883333,0.459917,378128.464,0.349263,0.000519,8,0.681492,-25.087,1,0.448377,3587,0.609022
15678,35.082,0.905281,0.566619,581799.061,0.543673,0.209238,6,0.416611,-10.246,1,0.040625,2797,0.869294
15679,59.693,0.113408,0.644554,151226.592,0.912201,0.000089,8,0.166336,-7.257,2,0.060308,1913,0.402700


In [42]:
d1 = y.drop_duplicates('music_genre').set_index(category)
dict_map = d1.to_dict()
print(dict_map['music_genre'])

{'Jazz': 4, 'Rap': 5, 'Rock': 6, 'Classical': 0, 'Electronic': 2, 'Country': 1, 'Hip-Hop': 3}


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("X_train shape : ",X_train.shape)
print("y_train shape : ",y_train.shape)

print("X_test shape : ",X_test.shape)
print("y_test shape : ",y_test.shape)

X_train shape :  (10976, 13)
y_train shape :  (10976, 1)
X_test shape :  (4705, 13)
y_test shape :  (4705, 1)


In [44]:
clf = CatBoostClassifier(
    iterations=400,
    max_depth=4,
    random_seed=42,
    learning_rate=0.2,
    custom_loss=['AUC', 'Accuracy'])

clf.fit(X_train, y_train,eval_set=(X_test, y_test),verbose=False)

print(f'Train accuracy: {clf.score(X_train,y_train)}')
print(f'Test accuracy: {clf.score(X_test,y_test)}')

Train accuracy: 0.8270772594752187
Test accuracy: 0.7846971307120085


In [45]:
X_test_final = pd.read_csv(path1)
X_test_final.head()



,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence
0,MSC_70753.0,"Bard, a Man of Lake-town - Extended Version",26.101,0.823765,0.115490,211462.287,0.125325,0.945272,Scale A Sharp,0.120392,-25.541,Both,Minor,0.038657,80.777,Duet,0.041238
1,MSC_24064.0,New Kings,66.325,0.015441,0.592838,249151.238,0.870798,0.000108,Scale E,0.406722,-3.905,Male,?,0.051368,76.004,Band,0.456309
2,MSC_22731.0,Silver Lining,71.871,0.001006,0.510340,215693.240,0.683077,0.000000,Scale C,0.084608,-6.825,Female,Major,0.034303,144.458,Solo,0.193332
3,MSC_32095.0,A Tale That Wasn't Right,56.151,0.315650,0.338336,307056.179,0.608904,0.000000,Scale A,0.276811,-8.382,Female,Minor,0.036934,118.145,Solo,0.214906
4,MSC_24198.0,Get Ready,43.687,0.038336,0.773904,352802.872,0.955500,0.000007,Scale D,0.301726,-4.733,Both,Major,0.079688,95.012,Duet,0.757347


In [46]:
X_test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instance_id        3921 non-null   object 
 1   track_name         3921 non-null   object 
 2   popularity         3755 non-null   float64
 3   acousticness       3921 non-null   float64
 4   danceability       3782 non-null   float64
 5   duration_ms        3900 non-null   float64
 6   energy             3888 non-null   float64
 7   instrumentalness   3898 non-null   float64
 8   key                3921 non-null   object 
 9   liveness           3921 non-null   float64
 10  loudness           3918 non-null   float64
 11  voice_gender       3735 non-null   object 
 12  mode               3907 non-null   object 
 13  speechiness        3914 non-null   float64
 14  tempo              3921 non-null   object 
 15  musician_category  3609 non-null   object 
 16  valence            3730 

In [47]:
print("shape of dataframe : ",X_test_final.shape)

shape of dataframe :  (3921, 17)


In [48]:
#X_test_final = X_test_final.dropna()
X_test_final

,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence
0,MSC_70753.0,"Bard, a Man of Lake-town - Extended Version",26.101,0.823765,0.115490,211462.287,0.125325,0.945272,Scale A Sharp,0.120392,-25.541,Both,Minor,0.038657,80.777,Duet,0.041238
1,MSC_24064.0,New Kings,66.325,0.015441,0.592838,249151.238,0.870798,0.000108,Scale E,0.406722,-3.905,Male,?,0.051368,76.004,Band,0.456309
2,MSC_22731.0,Silver Lining,71.871,0.001006,0.510340,215693.240,0.683077,0.000000,Scale C,0.084608,-6.825,Female,Major,0.034303,144.458,Solo,0.193332
3,MSC_32095.0,A Tale That Wasn't Right,56.151,0.315650,0.338336,307056.179,0.608904,0.000000,Scale A,0.276811,-8.382,Female,Minor,0.036934,118.145,Solo,0.214906
4,MSC_24198.0,Get Ready,43.687,0.038336,0.773904,352802.872,0.955500,0.000007,Scale D,0.301726,-4.733,Both,Major,0.079688,95.012,Duet,0.757347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3916,MSC_80955.0,"Violin Concerto No. 1 in A Minor, BWV 1041 : 2...",28.296,0.880373,0.239413,334617.187,0.046885,0.000008,Scale B,0.149835,-26.465,NaN,Major,0.044304,?,Solo,0.159485
3917,MSC_72767.0,Fly On the Wall,57.506,0.002973,0.658013,266627.734,-1.000000,0.000000,Scale E,0.308676,-6.282,Female,Minor,0.033955,120.048,Band,0.384031
3918,MSC_40192.0,"Aria mit 30 Veränderungen, BWV 988 ""Goldberg V...",40.085,1.065965,0.199430,237468.634,0.066235,0.998231,Scale G,0.074738,-30.092,NaN,Major,0.051459,66.095,Solo,0.333884
3919,MSC_56067.0,Buy Me A Rose,52.468,0.607947,0.677778,237945.370,0.313435,0.000002,Scale D Sharp,0.115475,-14.411,NaN,Major,0.032643,97.999,Band,0.203295


In [49]:
X_test_final = X_test_final.drop(columns=['track_name','voice_gender','musician_category'])

In [50]:
d = defaultdict(LabelEncoder)

def encode(col):  
    if col.dtype == 'object':
        return d[str(col)].fit_transform(col)
    else:
        return col 

column = X_test_final.iloc[:,0]
X_test_final = X_test_final.iloc[:,1:].apply(encode)
X_test_final['instance_id'] = column

In [51]:
X_test_final

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,instance_id
0,26.101,0.823765,0.115490,211462.287,0.125325,0.945272,1,0.120392,-25.541,2,0.038657,2594,0.041238,MSC_70753.0
1,66.325,0.015441,0.592838,249151.238,0.870798,0.000108,7,0.406722,-3.905,0,0.051368,2416,0.456309,MSC_24064.0
2,71.871,0.001006,0.510340,215693.240,0.683077,0.000000,3,0.084608,-6.825,1,0.034303,1512,0.193332,MSC_22731.0
3,56.151,0.315650,0.338336,307056.179,0.608904,0.000000,0,0.276811,-8.382,2,0.036934,634,0.214906,MSC_32095.0
4,43.687,0.038336,0.773904,352802.872,0.955500,0.000007,5,0.301726,-4.733,1,0.079688,3177,0.757347,MSC_24198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3916,28.296,0.880373,0.239413,334617.187,0.046885,0.000008,2,0.149835,-26.465,1,0.044304,3400,0.159485,MSC_80955.0
3917,57.506,0.002973,0.658013,266627.734,-1.000000,0.000000,7,0.308676,-6.282,2,0.033955,717,0.384031,MSC_72767.0
3918,40.085,1.065965,0.199430,237468.634,0.066235,0.998231,10,0.074738,-30.092,1,0.051459,2220,0.333884,MSC_40192.0
3919,52.468,0.607947,0.677778,237945.370,0.313435,0.000002,6,0.115475,-14.411,1,0.032643,3314,0.203295,MSC_56067.0


In [52]:
prediction = clf.predict(X_test_final.iloc[:,:13])
ids = (X_test_final.iloc[:,13].values)

In [53]:
prediction

array([[0],
       [6],
       [6],
       ...,
       [0],
       [1],
       [2]], dtype=int64)

In [54]:
inv_dict = {v: k for k, v in dict_map['music_genre'].items()}
inv_dict

{4: 'Jazz',
 5: 'Rap',
 6: 'Rock',
 0: 'Classical',
 2: 'Electronic',
 1: 'Country',
 3: 'Hip-Hop'}

In [55]:
theme = []
for val in prediction:
    theme.append(inv_dict[int(val)])


In [56]:
data = {'instance_id':ids,
        'music_genre': theme}
  
# Create DataFrame
df = pd.DataFrame(data)

df.to_csv('submission3.csv',index=False)
# Print the output.
df

,instance_id,music_genre
0,MSC_70753.0,Classical
1,MSC_24064.0,Rock
2,MSC_22731.0,Rock
3,MSC_32095.0,Rock
4,MSC_24198.0,Electronic
...,...,...
3916,MSC_80955.0,Classical
3917,MSC_72767.0,Rock
3918,MSC_40192.0,Classical
3919,MSC_56067.0,Country
